In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator
import numpy as np
import spacy
import random

from utils2 import translate_sentence, save_checkpoint, load_checkpoint

In [2]:
 torch.cuda.is_available()

True

In [3]:
import spacy.cli
import en_core_web_sm
import de_core_news_sm


spacy.cli.download("en_core_web_sm")
spacy.cli.download("de_core_news_sm")


spacy_ger = de_core_news_sm.load()
spacy_eng = en_core_web_sm.load()

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')


In [4]:
def tokenizer_de(text):
  return [tok.text for tok in spacy_ger.tokenizer(text)]

def tokenizer_eng(text):
  return [tok.text for tok in spacy_eng.tokenizer(text)]

In [5]:
tokenizer_eng("I ate my friends's O'neal apple yesterday")

['I', 'ate', 'my', 'friends', "'s", "O'neal", 'apple', 'yesterday']

In [6]:
german = Field(tokenize=tokenizer_de, lower=True, init_token="<sos>", eos_token="<eos>")

english = Field(
    tokenize=tokenizer_eng, lower=True, init_token="<sos>", eos_token="<eos>"
)

In [7]:
train_data, valid_data, test_data = Multi30k.splits(
    exts=(".de", ".en"), fields=(german, english)
)

In [8]:
german.build_vocab(train_data, max_size=10000, min_freq=2)
english.build_vocab(train_data, max_size=10000, min_freq=2)

In [9]:
#vocab.freqs in a python counter datatype
print("Five most common words in the dataset: " + str(english.vocab.freqs.most_common(5)))

Five most common words in the dataset: [('a', 49165), ('.', 27623), ('in', 14886), ('the', 10955), ('on', 8035)]


In [10]:

class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, p):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, bidirectional=True)

        self.fc_hidden = nn.Linear(hidden_size * 2, hidden_size)
        self.fc_cell = nn.Linear(hidden_size * 2, hidden_size)
        self.dropout = nn.Dropout(p)

    def forward(self, x):
        # x: (seq_length, N) where N is batch size

        embedding = self.dropout(self.embedding(x))
        # embedding shape: (seq_length, N, embedding_size)

        encoder_states, (hidden, cell) = self.rnn(embedding)
        # outputs shape: (seq_length, N, hidden_size*2)

        # Use forward, backward cells and hidden through a linear layer
        # so that it can be input to the decoder which is not bidirectional
        # Also using index slicing ([idx:idx+1]) to keep the dimension
        hidden = self.fc_hidden(torch.cat((hidden[0:1], hidden[1:2]), dim=2))
        cell = self.fc_cell(torch.cat((cell[0:1], cell[1:2]), dim=2))
        
        return encoder_states, hidden, cell


In [11]:
class Decoder(nn.Module):
    def __init__(
        self, input_size, embedding_size, hidden_size, output_size, num_layers, p
    ):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(hidden_size * 2 + embedding_size, hidden_size, num_layers)

        self.energy = nn.Linear(hidden_size * 3, 1)
        self.fc = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(p)
        self.softmax = nn.Softmax(dim=0)
        self.relu = nn.ReLU()

    def forward(self, x, encoder_states, hidden, cell):
        x = x.unsqueeze(0)
        # x: (1, N) where N is the batch size

        embedding = self.dropout(self.embedding(x))
        # embedding shape: (1, N, embedding_size)

        # encoder_states: (seq_length, N, hidden_size*2)
        sequence_length = encoder_states.shape[0]
        h_reshaped = hidden.repeat(sequence_length, 1, 1)
        # h_reshaped: (seq_length, N, hidden_size)

        energy = self.relu(self.energy(torch.cat((h_reshaped, encoder_states), dim=2)))
        # energy: (seq_length, N, 1)

        attention = self.softmax(energy)
        # attention: (seq_length, N, 1)

        # attention: (seq_length, N, 1), snk
        # encoder_states: (seq_length, N, hidden_size*2), snl
        # we want context_vector: (1, N, hidden_size*2), i.e knl
        context_vector = torch.einsum("snk,snl->knl", attention, encoder_states)

        rnn_input = torch.cat((context_vector, embedding), dim=2)
        # rnn_input: (1, N, hidden_size*2 + embedding_size)

        outputs, (hidden, cell) = self.rnn(rnn_input, (hidden, cell))
        # outputs shape: (1, N, hidden_size)

        predictions = self.fc(outputs).squeeze(0)
        # predictions: (N, hidden_size)

        return predictions, hidden, cell

In [12]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, source, target, teacher_force_ratio=0.5):
        batch_size = source.shape[1]
        target_len = target.shape[0]
        target_vocab_size = len(english.vocab)

        outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)
        encoder_states, hidden, cell = self.encoder(source)

        # First input will be <SOS> token
        x = target[0]

        for t in range(1, target_len):
            # At every time step use encoder_states and update hidden, cell
            output, hidden, cell = self.decoder(x, encoder_states, hidden, cell)

            # Store prediction for current time step
            outputs[t] = output

            # Get the best word the Decoder predicted (index in the vocabulary)
            best_guess = output.argmax(1)

            # With probability of teacher_force_ratio we take the actual next word
            # otherwise we take the word that the Decoder predicted it to be.
            # Teacher Forcing is used so that the model gets used to seeing
            # similar inputs at training and testing time, if teacher forcing is 1
            # then inputs at test time might be completely different than what the
            # network is used to. This was a long comment.
            x = target[t] if random.random() < teacher_force_ratio else best_guess

        return outputs

In [13]:
#Training Hyperparameters
num_epochs = 100
lr = 3e-4
batch_size = 32

In [14]:
#Model Hyperparameter
load_model = False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_size_encoder = len(german.vocab)
input_size_decoder = len(english.vocab)
output_size = len(english.vocab)
encoder_embedding_size = 300
decoder_embedding_size = 300
hidden_size = 1024
num_layers = 1
enc_dropout = 0.0
dec_dropout = 0.0

In [15]:
step = 0

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=batch_size,
    sort_within_batch=True,
    sort_key=lambda x: len(x.src),
    device=device,
)

In [18]:
next(iter(train_iterator)).trg

tensor([[   2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2,    2,    2,    2],
        [   4,    4,    4,    4,    4,    4,    4,    4,    4,  402,  388,    4,
           16,    4,    4,   16,    4,    4,    4,   16,    4,    4,    4,    9,
         3123,    4,    9,    7,   30,    4,    4,    4],
        [  14,    9,    9,   24,  348,    9,   38,   14,    9, 1071,   30,   38,
         3724,  122,   14,   50,   24, 2555,   14,   30,    9,    9,   38,    6,
         1827,  682,    6,   33,    6,    9,   87,   33],
        [  11,   10,  636,   33,   10,  137,   12,    6,    6,    0,    6,   12,
            6,   14,   36,   32,   14,  831,   56,   73,   45,   37,   12,  231,
           20,    9,  132,   10,   86,   32,   10,  189],
        [  55,   92,   73,   13,   92,   13,  127,    4,    4,    4,  890,   19,
         3664,   20,    6,    8,    6, 

In [27]:
encoder_net = Encoder(input_size_encoder, encoder_embedding_size, 
                      hidden_size, num_layers, enc_dropout).to(device)

decoder_net = Decoder(input_size_decoder, decoder_embedding_size, 
                      hidden_size, output_size, num_layers, dec_dropout).to(device)

In [28]:
model = Seq2Seq(encoder_net, decoder_net).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

In [106]:
pad_idx = english.vocab.stoi['<pad>']
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

In [107]:
if load_model:
    load_checkpoint(torch.load('my_checkpoint.pth.ptar'), model, optimizer)

In [108]:
sentence = "ein boot mit mehreren männern darauf wird von einem großen pferdegespann ans ufer gezogen."

for epoch in range(num_epochs):
    
    print(f'Epoch [{epoch} / {num_epochs}]')

    checkpoint = {'state_dict': model.state_dict(), optimizer: optimizer.state_dict()}
    save_checkpoint(checkpoint)

    model.eval()
    translated_sentence = translate_sentence(model, sentence, german, english, device, max_length=50)

    print(f"Translated example sentence: \n {translated_sentence}")

    model.train()

    for batch_idx, batch in enumerate(train_iterator):
        
        inp_data = batch.src.to(device)
        target = batch.trg.to(device)

        output = model(inp_data, target)
        output = output[1:].reshape(-1, output.shape[2])
        target = target[1:].reshape(-1)

        optimizer.zero_grad()
        loss = criterion(output, target)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
        optimizer.step()

        step = step + 1

Epoch [0 / 100]
=> Saving checkpoint
Hreshapedtorch.Size([17, 1, 1024])
torch.Size([17, 1, 2048])
Hreshapedtorch.Size([17, 1, 1024])
torch.Size([17, 1, 2048])
Hreshapedtorch.Size([17, 1, 1024])
torch.Size([17, 1, 2048])
Hreshapedtorch.Size([17, 1, 1024])
torch.Size([17, 1, 2048])
Hreshapedtorch.Size([17, 1, 1024])
torch.Size([17, 1, 2048])
Hreshapedtorch.Size([17, 1, 1024])
torch.Size([17, 1, 2048])
Hreshapedtorch.Size([17, 1, 1024])
torch.Size([17, 1, 2048])
Hreshapedtorch.Size([17, 1, 1024])
torch.Size([17, 1, 2048])
Hreshapedtorch.Size([17, 1, 1024])
torch.Size([17, 1, 2048])
Hreshapedtorch.Size([17, 1, 1024])
torch.Size([17, 1, 2048])
Hreshapedtorch.Size([17, 1, 1024])
torch.Size([17, 1, 2048])
Hreshapedtorch.Size([17, 1, 1024])
torch.Size([17, 1, 2048])
Hreshapedtorch.Size([17, 1, 1024])
torch.Size([17, 1, 2048])
Hreshapedtorch.Size([17, 1, 1024])
torch.Size([17, 1, 2048])
Hreshapedtorch.Size([17, 1, 1024])
torch.Size([17, 1, 2048])
Hreshapedtorch.Size([17, 1, 1024])
torch.Size([1

KeyboardInterrupt: 

In [ ]:
translate_sentence(model, "Diese Jungs schauen sich jeden Tag die Nachrichten an", german, english, device, max_length=50)